In [13]:
import matplotlib.pyplot as plt
import numpy as  np
from scipy.interpolate import CubicSpline 
import pandas as pd
import plotly.graph_objects as go
from scipy.stats import norm

from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True) 

In [14]:
### Funtion to interpolate 
x  = np.linspace(-10, 10, 8)
y  = np.linspace(-10, 10, 10)
xx,  yy  = mesh = np.meshgrid(x, y)
real_values = xx**2 + yy**2

In [15]:
real_values.shape

(10, 8)

In [16]:
Splines = {}   
for u in x:
    i = np.where(x==u)[0][0]
    Splines[u] = CubicSpline(y, real_values[:,i])
    

In [17]:
Splines

{-10.0: <scipy.interpolate._cubic.CubicSpline at 0x1366cfef0>,
 -7.142857142857142: <scipy.interpolate._cubic.CubicSpline at 0x1366e3450>,
 -4.285714285714286: <scipy.interpolate._cubic.CubicSpline at 0x1366e34a0>,
 -1.4285714285714288: <scipy.interpolate._cubic.CubicSpline at 0x1366cfcc0>,
 1.4285714285714288: <scipy.interpolate._cubic.CubicSpline at 0x1344b4770>,
 4.2857142857142865: <scipy.interpolate._cubic.CubicSpline at 0x1366e3590>,
 7.142857142857142: <scipy.interpolate._cubic.CubicSpline at 0x1366e3540>,
 10.0: <scipy.interpolate._cubic.CubicSpline at 0x1366e35e0>}

In [18]:
fig = go.Figure(data=[go.Surface(z=real_values, x=xx, y=yy)])

fig.update_layout(title='Surface', autosize=True,
                  width=700, height=700,
                  scene=dict(
        xaxis_title='log(S/K)',
        yaxis_title='Volaility (log)',
        zaxis_title='Surface'))

fig.show()

In [19]:
class Spline2d:
    def __init__(self,x1, x2, r_values):
        self.x1 = x1
        self.x2 = x2
        self.r_values = r_values

    
    def computes_spline(self, x1_val, x2_val):
        Splines = {}
        for u in self.x1:
            i = np.where (self.x1 ==u)[0][0]
            Splines[u] = CubicSpline(self.x2, self.r_values[:,i])
        t = [Splines[u](x2_val)  for u in self.x1]
        Spline_f = CubicSpline(self.x1, t)
        return Spline_f(x1_val)
        
    

        

In [20]:
S0 = 100
vol_ref = .2
r = 0.03
T = 1

In [21]:
def black_scholes_price(x1, T, S, x2, r=0.03):
    vol_ref = 0.2
    vol = vol_ref * np.exp(x2)
    
    d_plus = (x1 + r*T) / (vol*np.sqrt(T)) + 1/2 * vol * np.sqrt(T)
    d_minus = (x1 + r*T) / (vol*np.sqrt(T)) - 1/2 * vol * np.sqrt(T)
    
    return S * norm.cdf(d_plus) - S * np.exp(-x1) * np.exp(-r*T) * norm.cdf(d_minus)

In [22]:
def delta(x1, T, S, x2, r=0.03):
    vol_ref = 0.2
    vol = vol_ref * np.exp(x2)
    
    d_plus = (x1 + r*T) / (vol*np.sqrt(T)) + 1/2 * vol * np.sqrt(T)
    d_minus = (x1 + r*T) / (vol*np.sqrt(T)) - 1/2 * vol * np.sqrt(T)
    
    return (S/vol*np.sqrt(T))* norm.pdf(d_plus) - S  * np.exp(-r*T) * np.exp(-x1)*((1/(vol*np.sqrt(T)))*norm.pdf(d_minus)  -norm.cdf(d_minus) )

In [23]:
def vega(x1, T, S, x2, r=0.03):
    vol_ref = 0.2
    vol = vol_ref * np.exp(x2)
    
    d_plus = (x1 + r*T) / (vol*np.sqrt(T)) + 1/2 * vol * np.sqrt(T)
    d_minus = (x1 + r*T) / (vol*np.sqrt(T)) - 1/2 * vol * np.sqrt(T)
    
    return 0.5*S*(np.exp(-x1)*d_plus*norm.pdf(d_minus) -d_minus*norm.pdf(d_plus) )

In [24]:
class Spline2d:
    def __init__(self,x1, x2, r_values):
        self.x1 = x1
        self.x2 = x2
        self.r_values = r_values

    
    def computes_spline(self, x1_val, x2_val):
        Splines = {}
        for u in self.x2:
            i = np.where (self.x2 ==u)[0][0]
            Splines[u] = CubicSpline(self.x1, self.r_values[i,:], bc_type=((2, 0), (1, delta(self.x1[len(self.x1)-1], T, S0, u, r=0.03))))
        t = [Splines[u](x1_val)  for u in self.x2]
        Spline_f = CubicSpline(self.x2, t, bc_type=((2, 0), (1, vega(x1_val, T, S0, self.x2[len(self.x2)-1], r=0.03))))
        return Spline_f(x2_val)

In [25]:
x2 = np.linspace(-6,6, 10)
x1 = np.linspace(-6, 6,10)
x1 = np.append(x1, 8)
x1 = np.insert(x1, 0,-8)
x2 = np.append(x2, 8)
x2 = np.insert(x2, 0,-8)
xx , yy = np.meshgrid(x1, x2)
prices = black_scholes_price(xx, T, S0, yy, r=0.03)
fig = go.Figure(data=[go.Surface(z=prices, x=xx, y=yy)])

fig.update_layout(title='Surface', autosize=True,
                  width=700, height=700,
                  scene=dict(
        xaxis_title='log(S/K)',
        yaxis_title='Volaility (log)',
        zaxis_title='Surface'))

In [29]:
x2

array([-8.        , -6.        , -4.66666667, -3.33333333, -2.        ,
       -0.66666667,  0.66666667,  2.        ,  3.33333333,  4.66666667,
        6.        ,  8.        ])

In [30]:
approx = Spline2d(x1, x2, prices)
x, y = np.linspace(-15, 15 ,20), np.linspace(-15, 15 ,20)
#xx, yy = np.meshgrid(x, y)
surface_spline = np.zeros((len(y), len(x)))
for j in range (len(x)):
    for i in range(len(y)): 
        surface_spline[i, j] = approx.computes_spline(x[j],y[i])
    

In [31]:
fig = go.Figure(data=[go.Surface(z=surface_spline, x=x, y=y)])

fig.update_layout(title='Surface', autosize=True,
                  width=700, height=700,
                  scene=dict(
        xaxis_title='log(S/K)',
        yaxis_title='Volaility (log)',
        zaxis_title='Surface'))

In [353]:
class Spline2d:
    def __init__(self,x1, x2):
        self.x1 = x1
        self.x2 = x2

    
    def computes_spline_1D(self, x1_val, x2_val):
        Splines = {}
        xx1, xx2 = np.meshgrid(self.x1, self.x2)
        
        r_values  = black_scholes_price(xx1, T, S0, xx2, r=0.03)
        
        for i, u in enumerate(self.x1):
            Splines[u] = CubicSpline(self.x2, r_values[:, i], bc_type=((2, 0), (2, 0)))

        t = np.array([Splines[u](x2_val)  for u in self.x1])
        
        Spline_f = CubicSpline(self.x1, t, bc_type=((2, 0), (2, 0)))
        
        return Spline_f(x1_val)
    
    def computes_spline(self, x1_val, x2_val):
        #assert len(x1_val) == len(x2_val)
        surface_spline = np.zeros((len(x2_val), len(x1_val)))
        
        for i in range(len(x2_val)): 
            surface_spline[i, :] = self.computes_spline_1D(x1_val, x2_val[i])
            
        return surface_spline
        

In [368]:
x1 = np.linspace(-6, 6, 20)
x2 = np.linspace(-6, 6, 10)

x1 = np.append(x1, 8)
x1 = np.insert(x1, 0,-8)
x2 = np.append(x2, 8)
x2 = np.insert(x2, 0,-8)

approx = Spline2d(x1, x2)

x = np.linspace(-10, 10, 100)
y = np.linspace(-10, 10, 100)

surface_spline = approx.computes_spline(x, y)
    

In [369]:
fig = go.Figure(data=[go.Surface(z=surface_spline, x=x, y=y)])

fig.update_layout(title='Surface', autosize=True,
                  width=700, height=700,
                  scene=dict(
        xaxis_title='log(S/K)',
        yaxis_title='Volaility (log)',
        zaxis_title='Surface'))